# Project Milestone 2: Cleaning/Formatting Flat File Source

### Load Data and Necessary Packages

In [1]:
import pandas as pd
import warnings
import pycountry_convert as pc

warnings.filterwarnings('ignore')                 # some warnings are repressed
csv_df = pd.read_csv('worldcitiespop.csv')

### View Raw Data

In [2]:
csv_df.head(5)

,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,6.0,NaN,42.483333,1.466667
1,ad,aixirivali,Aixirivali,6.0,NaN,42.466667,1.500000
2,ad,aixirivall,Aixirivall,6.0,NaN,42.466667,1.500000
3,ad,aixirvall,Aixirvall,6.0,NaN,42.466667,1.500000
4,ad,aixovall,Aixovall,6.0,NaN,42.466667,1.483333


Upon initial observation, there are some obvious transformations that need to take place given the use of this data. The first such observation is in relation to the *Country* column, in which country codes are given rather than full country names. Because our Wikipedia source contains full names, and this column is used for joining, we much convert these country codes to full country names. Additionally, we can remove the *City* column, as the *AccentCity* column is the same, just capitalized and including any accents. We will then rename the *AccentCity* column to *City*. One final observation is that duplicates exist in the dataset, in which various spellings are present for the same city. Using latitude and longitude as a guide, dupliactes will be removed. Additional transformations may be performed along the way

### Step 1: Drop Columns - *City* and *Region*

In [3]:
# drops city and region columns from dataframe
csv_df = csv_df.drop(['City'], axis = 1)
csv_df = csv_df.drop(['Region'], axis = 1)

In [4]:
csv_df.head(5)

,Country,AccentCity,Population,Latitude,Longitude
0,ad,Aixàs,NaN,42.483333,1.466667
1,ad,Aixirivali,NaN,42.466667,1.500000
2,ad,Aixirivall,NaN,42.466667,1.500000
3,ad,Aixirvall,NaN,42.466667,1.500000
4,ad,Aixovall,NaN,42.466667,1.483333


We are first removing the *City* column, given that it is a duplicate of the *AccentCity* column, just without accents and capitalization. We will also remove the *Region* column, since it is not going to be used in the final analysis.

### Step 2: Rename Column - *AccentCity* to *City*

In [5]:
# rename AccentCity column to City
csv_df = csv_df.rename(columns = {"AccentCity": "City"})

In [6]:
csv_df.head(5)

,Country,City,Population,Latitude,Longitude
0,ad,Aixàs,NaN,42.483333,1.466667
1,ad,Aixirivali,NaN,42.466667,1.500000
2,ad,Aixirivall,NaN,42.466667,1.500000
3,ad,Aixirvall,NaN,42.466667,1.500000
4,ad,Aixovall,NaN,42.466667,1.483333


Now that our dataset only contains one column for the city name, we will rename the *AccentCity* column to *City* for clarity and ease of use.

### Step 3: Remove NaN's from Population Column

In [7]:
csv_df = csv_df[csv_df['Population'].notna()]

In [8]:
csv_df.head(5)

,Country,City,Population,Latitude,Longitude
6,ad,Andorra la Vella,20430.0,42.500000,1.516667
20,ad,Canillo,3292.0,42.566667,1.600000
32,ad,Encamp,11224.0,42.533333,1.583333
49,ad,La Massana,7211.0,42.550000,1.516667
53,ad,Les Escaldes,15854.0,42.500000,1.533333


Given that population size is going to be a key factor in this analysis, those cities not containing a population size will be removed. Further, given that population information is more readily available for more well-known and populous cities, this step also removes small cities that may not contain much relevant data.

### Step 4: Convert Country Codes to Country Names

In [9]:
# function to convert country codes to country names 
def convert_to_name(row):
    try: 
        # uses convert (extension of pycountry) to covert code to name (code must be uppercase)
        cn = pc.country_alpha2_to_country_name(row['Country'].upper())
    except (KeyError):
        # if country code is not identified, input 'Not Found' for further analysis
        cn = "Not Found"
    return cn

In [10]:
# use lambda function to apply conversion function to each row and save as new column 
csv_df['Country_Name'] = csv_df.apply(lambda row: convert_to_name(row), axis = 1)

In [11]:
# find county codes that could not be converted
print(csv_df[csv_df["Country_Name"].str.contains("Not Found")].Country.unique())

['an']


In [12]:
# change all country names of the country code 'an' to correct country
csv_df.loc[csv_df.Country == 'an', 'Country_Name'] = "Netherlands Antilles"

In [13]:
# now that we have fixed all the country names, we can drop the original country column 
csv_df = csv_df.drop(['Country'], axis = 1)

In [14]:
# we will rename the country_name column to just be country
csv_df = csv_df.rename(columns = {"Country_Name": "Country"})

In [15]:
csv_df.head(5)

,City,Population,Latitude,Longitude,Country
6,Andorra la Vella,20430.0,42.500000,1.516667,Andorra
20,Canillo,3292.0,42.566667,1.600000,Andorra
32,Encamp,11224.0,42.533333,1.583333,Andorra
49,La Massana,7211.0,42.550000,1.516667,Andorra
53,Les Escaldes,15854.0,42.500000,1.533333,Andorra


Because we will be joining the Wikipedia suicide data on country name, we need to convert country codes to country name. By using the python package pycountry_covert, we easily convert these codes to names, with only **AN** left to manually update.

### Step 5: Remove Duplicate Cities

In [16]:
# store the number of rows before duplicates are removed
length_before = len(csv_df)

In [17]:
# remove all instances where latitude and longitude are duplicated, keeping the first instance
csv_df = csv_df.drop_duplicates(subset = ["Latitude", "Longitude"], keep = 'first')

In [18]:
# calculate number of rows before and after to see how many rows were removed
diff = length_before - len(csv_df)
print("Duplicates Removed: ", diff)

Duplicates Removed:  138


Some cities appear twice due to varied spelling of the city name. These duplicates are identified through the longitude and latitude, and only the first instance is kept.

### Step 6: Re-Index DataFrame

In [19]:
# resets the dataframe index
csv_df = csv_df.reset_index()
# drops additional index column that is created when the above function runs
csv_df = csv_df.drop(['index'], axis = 1)

In [20]:
csv_df.head(5)

,City,Population,Latitude,Longitude,Country
0,Andorra la Vella,20430.0,42.500000,1.516667,Andorra
1,Canillo,3292.0,42.566667,1.600000,Andorra
2,Encamp,11224.0,42.533333,1.583333,Andorra
3,La Massana,7211.0,42.550000,1.516667,Andorra
4,Les Escaldes,15854.0,42.500000,1.533333,Andorra


Since many rows have been removed in the process of cleaning the data, the indexing is now incorrect. We reset the index to be able to accuractely access elements by index if needed in future analysis.

### View Cleaned Data

In [21]:
csv_df.head(20)

,City,Population,Latitude,Longitude,Country
0,Andorra la Vella,20430.0,42.500000,1.516667,Andorra
1,Canillo,3292.0,42.566667,1.600000,Andorra
2,Encamp,11224.0,42.533333,1.583333,Andorra
3,La Massana,7211.0,42.550000,1.516667,Andorra
4,Les Escaldes,15854.0,42.500000,1.533333,Andorra
5,Ordino,2553.0,42.550000,1.533333,Andorra
6,Sant Julià de Lòria,8020.0,42.466667,1.500000,Andorra
7,Abu Dhabi,603687.0,24.466667,54.366667,United Arab Emirates
8,Dubai,1137376.0,25.258172,55.304717,United Arab Emirates
9,Sharjah,543942.0,25.357310,55.403304,United Arab Emirates


In [22]:
csv_df.tail(20)

,City,Population,Latitude,Longitude,Country
47822,Gwanda,20731.0,-20.933333,29.000000,Zimbabwe
47823,Gweru,201879.0,-19.450000,29.816667,Zimbabwe
47824,Harare,2213701.0,-17.817778,31.044722,Zimbabwe
47825,Hwange,41632.0,-18.366667,26.483333,Zimbabwe
47826,Kadoma,100276.0,-18.350000,29.916667,Zimbabwe
47827,Kariba,34797.0,-16.516667,28.800000,Zimbabwe
47828,Karoi,24981.0,-16.816667,29.683333,Zimbabwe
47829,Kwekwe,116332.0,-18.916667,29.816667,Zimbabwe
47830,Marondera,82294.0,-18.183333,31.550000,Zimbabwe
47831,Masvingo,86772.0,-20.083333,30.833333,Zimbabwe
